In [1]:
import sys
sys.path.append('../../')
sys.path.append('../')
import time
import pnn

In [2]:
import numpy as np
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.fx import subgraph_rewriter, symbolic_trace
import utils
from torch.fx import Proxy, Graph, GraphModule
from torch.fx.passes.utils.matcher_utils import SubgraphMatcher

replace success!


In [3]:
from torch.profiler import profile, record_function, ProfilerActivity
import time
import torch._dynamo as dynamo


In [4]:
def calculate_mean_and_variance_manual(data):
    n = len(data)
    mean = sum(data) / n
    variance = sum((x - mean) ** 2 for x in data) / n
    return mean, variance

In [4]:
def gen_pattern_replace_and_matcher_for_loop_pnn(traced,
                                                  redundancy_part_slice,non_redundancy_part_slice,
                                                  embed_node_name,getitem_node_names,num_field,batch = 4096,match_func = None
                                                ):
  from torch.fx.passes.utils.matcher_utils import SubgraphMatcher


  def _match(match,ori,pat):
    return True 
  env  = utils.get_env(traced)
  target_node = env[embed_node_name]
  target_node_mod = utils.get_target_mod(traced,target_node.target)
  shape_info = target_node_mod.weight.data.shape
  getitem_node_args = [env[i].args[1] for i in getitem_node_names]
  class PatternClass(torch.nn.Module):
      def __init__(self):
          super().__init__()

      def pn(self,x):
         return torch.sum(x[getitem_node_args[0]] * x[getitem_node_args[1]], dim = 2)

      def forward(self,x):
          return self.pn(x)
  pattern = PatternClass()  
  pattern_trace = symbolic_trace(pattern)
  pattern_graph = pattern_trace.graph
  original_graph = traced.graph
  matcher =  SubgraphMatcher(pattern_graph, match_output=False, match_placeholder=False,
                              remove_overlapping_matches=True)
  _matches = matcher.match(original_graph)
  match_filters = [_match if match_func is None else match_func]
  _matches = [
      m for m in _matches
      if all(match_filter(m, original_graph, pattern_graph)
              for match_filter in match_filters)
  ]  
  # 因为在过滤器中做了限制应该只有一个符合要求的
  _matched = _matches[0]
  pattern_env = utils.get_env(pattern_trace)
  node_map = _matched.nodes_map
  
  
#   linear_node = node_map[pattern_env['mlp']]
#   linear_node_module = utils.get_target_mod(traced,linear_node.target)
#   linear_node_weight = linear_node_module.weight.data
#   linear_node_bias = linear_node_module.bias.data
  
  class ReplacementClass(torch.nn.Module):
    def __init__(self):
      super().__init__()
      self.num_fields = num_field
      self.num_prefix = redundancy_part_slice[1].stop
      # self.ori_linear_shape = linear_node_weight.shape
      self.num_sufix = self.num_fields - self.num_prefix
      self.total = self.num_fields * (self.num_fields - 1) // 2
      self.redundancy_cross_part_total = self.num_prefix * (self.num_prefix - 1) // 2
      self.non_redundancy_cross_part_total = self.num_sufix * (self.num_sufix - 1) // 2
      self.rest_cross_part_total = self.total - self.redundancy_cross_part_total - self.non_redundancy_cross_part_total

    def pn(self,reducey_x,non_redundancy_x):
       redundancy_x_row, reducey_x_col = list(),list()
       non_redundancy_x_row, non_redundancy_x_col = list(), list()
       mixed_x_row, mixed_x_col = list(), list()
       prefix = self.num_prefix
       sufix = self.num_sufix
       for i in range(prefix - 1):
          for j in range(i+1,prefix):
             redundancy_x_row.append(i),reducey_x_col.append(j)
       for i in range(sufix - 1):
          for j in range(i + 1,sufix):
             non_redundancy_x_row.append(i),non_redundancy_x_col.append(j)

       for i in range(prefix):
          for j in range(sufix):
             mixed_x_row.append(i),mixed_x_col.append(j)
       return torch.sum((reducey_x[redundancy_x_row] * reducey_x[reducey_x_col]).unsqueeze(0),dim = -1),\
              torch.sum(non_redundancy_x[:,non_redundancy_x_row] * non_redundancy_x[:,non_redundancy_x_col],dim= -1),\
              torch.sum(reducey_x[mixed_x_row] * non_redundancy_x[:,mixed_x_col],dim = -1)


    def forward(self,x):
      redundancy_part_embed = x[redundancy_part_slice]
      non_redundancy_part_embed = x[non_redundancy_part_slice]
      redundancy_part_pn, non_redundancy_part_pn, mixed_part_pn = self.pn(redundancy_part_embed,non_redundancy_part_embed)
      
      redundancy_part_pn = redundancy_part_pn.repeat(batch,1)
      cross_term =  torch.concat([redundancy_part_pn,non_redundancy_part_pn,mixed_part_pn], dim = 1)

      return cross_term
    
  
  return pattern,ReplacementClass(),_match

In [5]:
# 只修改哈达玛积
def workload_pnn(num_field, prefix,dim = 64,l = [1024,512,256],batch = 4096):
  print(f"now gen workload of PNN with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}")
  pnn_loop = pnn.ProductNeuralNetworkModel([100 for i in range(num_field)],dim,l,0.1)

  pnn_model_traced_ori = symbolic_trace(pnn_loop)
  
  pnn_model_modify = pnn.ProductNeuralNetworkModel([100 for i in range(num_field)],dim,l,0.1)
  pnn_model_traced_modify = symbolic_trace(pnn_model_modify)
  pattern,replace,match = gen_pattern_replace_and_matcher_for_loop_pnn(pnn_model_traced_modify,
                                                                      (0,slice(None,prefix,None)),(slice(None,None,None),slice(prefix,None,None)),
                                                                      embed_node_name = "embedding_embedding",
                                                                      getitem_node_names = ["getitem","getitem_1"],num_field=num_field,batch = batch)
  matches = subgraph_rewriter.replace_pattern_with_filters(pnn_model_traced_modify, pattern, replace,[match])
  return pnn_model_traced_ori,pnn_model_traced_modify

In [9]:
def gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32, workload_func = workload_pnn,l = [1024,512,256]):
  def run(model):
    traced_model = torch.jit.trace(model, torch.randint(low=0, high=88, size=(batch,num_field), dtype=torch.long))
    compiled_model = torch.compile(traced_model, backend="inductor")
    compiled_model.eval()
    total_time = []
    t = torch.randint(low=0, high=88, size=(batch ,num_field), dtype=torch.long)

    with profile(activities=[ProfilerActivity.CPU], record_shapes=True) as prof:
        with record_function("model_inference"):
          for i in range(100):
              start_time = time.time()  # 开始计时
              with torch.no_grad():
                soutput = compiled_model(t)
              end_time = time.time()  # 结束计时
              
              # 计算并打印函数执行所需的时间
              elapsed_time = end_time - start_time
              total_time.append(elapsed_time * 1000)
    print(calculate_mean_and_variance_manual(total_time))
    return prof
  print(f"now gen workload of DFM with config: dim: {dim}, num_field: {num_field}, prefix: {prefix}, batch :{batch}")
  ori, modify = workload_func(num_field,prefix,dim,l=l,batch = batch)
  p1 = run(ori)
  p2 = run(modify)
  return p1, p2

In [12]:
def genWorkload(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64):
  ori_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn/pnn_{batch}_{num_field}_{prefix}_{dim}_ori.onnx'
  modify_model_name = f'/home/yssun/pytorch-fm/torchfm/model/test_fx/exp/model_repo/pnn/pnn_{batch}_{num_field}_{prefix}_{dim}_modify.onnx'
  ori, modify = workload_pnn(num_field,prefix,dim,batch = batch)
  torch.onnx.export(ori,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  ori_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )
  torch.onnx.export(modify,               # 模型 being run
                  torch.randint(low=0, high=20, size=(batch,num_field), dtype=torch.long),                  # 模型输入 (or a tuple for multiple inputs)
                  modify_model_name,        # 导出文件的文件名
                  export_params=True, # 如果设置为True，则参数也会被导出。注意某些情况下参数可能无法被导出。
                  opset_version=10,   # ONNX版本
                  do_constant_folding=True,  # 是否执行常量折叠以优化模型
                  input_names = ['input'],   # 输入的名称
                  output_names = ['output'], # 输出的名称
                  )

In [13]:
dims= [32]
batches = [1024,2048,4096]
num_field_and_prefixs = [(34 * 5,29*5),(22 * 5,10 * 5),(34,29),(22 ,10 )]
for dim in dims:
  for batch in batches:
    for num_field,prefix in num_field_and_prefixs:
      genWorkload(num_field=num_field,prefix=prefix,batch=batch,dim=dim)

now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


...

/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/tensorrt/lib/python3.9/site-packages/torch/onnx/symbolic_opset9.py:1173: UserWarning: This model contains a squeeze operation on dimension 1. If the model is intended to be used with dynamic input shapes, please use opset version 11 to export the model.
  warnings.warn(


In [37]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1017 / 1024 (99.3%)
Greatest absolute difference: 0.004365146160125732 at index (31,) (up to 1e-05 allowed)
Greatest relative difference: 0.00862310039521317 at index (31,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:34 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:34 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:34 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(8.237698078155518, 7.175969275402849)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1022 / 1024 (99.8%)
Greatest absolute difference: 0.004552960395812988 at index (227,) (up to 1e-05 allowed)
Greatest relative difference: 0.009130312239801161 at index (227,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:35 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:36 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:36 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.4291558265686035, 0.551365407608273)


(<torch.profiler.profiler.profile at 0x7f57a5fb3670>,
 <torch.profiler.profiler.profile at 0x7f58209571c0>)

In [38]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2026 / 2048 (98.9%)
Greatest absolute difference: 0.005121380090713501 at index (704,) (up to 1e-05 allowed)
Greatest relative difference: 0.01050987045298034 at index (704,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:37 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:40 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:40 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(32.923901081085205, 9.637103118365076)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2031 / 2048 (99.2%)
Greatest absolute difference: 0.00512126088142395 at index (1857,) (up to 1e-05 allowed)
Greatest relative difference: 0.010163829280837556 at index (1857,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:41 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:42 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:42 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(9.682245254516602, 3.6260778905216284)


(<torch.profiler.profiler.profile at 0x7f5768475910>,
 <torch.profiler.profiler.profile at 0x7f57105bd550>)

In [39]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4070 / 4096 (99.4%)
Greatest absolute difference: 0.004721015691757202 at index (3168,) (up to 1e-05 allowed)
Greatest relative difference: 0.009609785479469822 at index (3168,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:43 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:49 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:49 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(58.144052028656006, 6.124738574789035)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4064 / 4096 (99.2%)
Greatest absolute difference: 0.005401164293289185 at index (2404,) (up to 1e-05 allowed)
Greatest relative difference: 0.010797716591531342 at index (2404,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:50 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:39:54 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:39:54 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(31.747872829437256, 3.7245110773426404)


(<torch.profiler.profiler.profile at 0x7f5708958ca0>,
 <torch.profiler.profiler.profile at 0x7f56e8268f10>)

In [40]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 22, prefix: 10, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8107 / 8192 (99.0%)
Greatest absolute difference: 0.005329132080078125 at index (6445,) (up to 1e-05 allowed)
Greatest relative difference: 0.010538407880986523 at index (6445,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:39:55 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:07 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:07 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(115.58759212493896, 7.251470219102885)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8097 / 8192 (98.8%)
Greatest absolute difference: 0.005121499300003052 at index (7585,) (up to 1e-05 allowed)
Greatest relative difference: 0.010167665393408404 at index (7585,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:08 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:19 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:19 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(104.29039478302002, 6.411856153113149)


(<torch.profiler.profiler.profile at 0x7f573816e400>,
 <torch.profiler.profiler.profile at 0x7f56a82273a0>)

In [41]:
gen_and_test(num_field = 22,prefix = 10, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1010 / 1024 (98.6%)
Greatest absolute difference: 0.004457354545593262 at index (888,) (up to 1e-05 allowed)
Greatest relative difference: 0.008592251577339168 at index (888,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:20 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:23 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:23 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(32.0242977142334, 3.230932433689304)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1018 / 1024 (99.4%)
Greatest absolute difference: 0.005352199077606201 at index (130,) (up to 1e-05 allowed)
Greatest relative difference: 0.010388424938004374 at index (130,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:24 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:24 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:24 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(6.814675331115723, 1.0370986119369263)


(<torch.profiler.profiler.profile at 0x7f568816c6a0>,
 <torch.profiler.profiler.profile at 0x7f56682b4460>)

In [42]:
gen_and_test(num_field = 22,prefix = 10, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2026 / 2048 (98.9%)
Greatest absolute difference: 0.0040170252323150635 at index (286,) (up to 1e-05 allowed)
Greatest relative difference: 0.008130483925119612 at index (286,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:26 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:32 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:32 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(57.95710325241089, 3.6397596583071845)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2035 / 2048 (99.4%)
Greatest absolute difference: 0.005096137523651123 at index (940,) (up to 1e-05 allowed)
Greatest relative difference: 0.010321542754291835 at index (940,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:33 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:37 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:37 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(39.696693420410156, 1.8374022965872427)


(<torch.profiler.profiler.profile at 0x7f56b85bf400>,
 <torch.profiler.profiler.profile at 0x7f56141afe50>)

In [43]:
gen_and_test(num_field = 22,prefix = 10, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4050 / 4096 (98.9%)
Greatest absolute difference: 0.00477224588394165 at index (1714,) (up to 1e-05 allowed)
Greatest relative difference: 0.009602504410867494 at index (1714,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:38 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:49 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:49 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(108.8698148727417, 5.815583025992055)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4053 / 4096 (99.0%)
Greatest absolute difference: 0.004994034767150879 at index (2375,) (up to 1e-05 allowed)
Greatest relative difference: 0.009799989496585865 at index (2375,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:40:51 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:40:58 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:40:58 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(78.49224328994751, 2.172531192235283)


(<torch.profiler.profiler.profile at 0x7f521a9d6e50>,
 <torch.profiler.profiler.profile at 0x7f52199addf0>)

In [44]:
gen_and_test(num_field = 22,prefix = 10, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 22, prefix: 10, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 22, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8130 / 8192 (99.2%)
Greatest absolute difference: 0.005773305892944336 at index (4125,) (up to 1e-05 allowed)
Greatest relative difference: 0.011303576894723274 at index (4125,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:41:00 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:41:26 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:41:26 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(251.59627676010132, 23.966966314054588)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8129 / 8192 (99.2%)
Greatest absolute difference: 0.0054669976234436035 at index (4463,) (up to 1e-05 allowed)
Greatest relative difference: 0.010804788971413445 at index (4463,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:41:27 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:41:46 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:41:46 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(185.4892635345459, 32.99732625864635)


(<torch.profiler.profiler.profile at 0x7f52181698b0>,
 <torch.profiler.profiler.profile at 0x7f5216c91580>)

In [45]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1018 / 1024 (99.4%)
Greatest absolute difference: 0.004462718963623047 at index (260,) (up to 1e-05 allowed)
Greatest relative difference: 0.008609620761338305 at index (260,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:41:48 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:41:52 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:41:52 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(37.281527519226074, 0.7397533156790814)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1020 / 1024 (99.6%)
Greatest absolute difference: 0.004677772521972656 at index (867,) (up to 1e-05 allowed)
Greatest relative difference: 0.009100937960200095 at index (867,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:41:53 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:41:53 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:41:53 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5.522923469543457, 0.35597371991116233)


(<torch.profiler.profiler.profile at 0x7f521563aca0>,
 <torch.profiler.profiler.profile at 0x7f5213e50c10>)

In [46]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2034 / 2048 (99.3%)
Greatest absolute difference: 0.0053334832191467285 at index (1180,) (up to 1e-05 allowed)
Greatest relative difference: 0.01043424174823137 at index (1180,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:41:54 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:42:01 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:42:01 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(68.94083023071289, 1.3708961137126607)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2027 / 2048 (99.0%)
Greatest absolute difference: 0.0044152140617370605 at index (1294,) (up to 1e-05 allowed)
Greatest relative difference: 0.008589331951155313 at index (1294,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:42:02 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:42:04 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:42:04 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(18.456721305847168, 0.8079630259771875)


(<torch.profiler.profiler.profile at 0x7f56501b0310>,
 <torch.profiler.profiler.profile at 0x7f5210f767f0>)

In [47]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4066 / 4096 (99.3%)
Greatest absolute difference: 0.005644410848617554 at index (3652,) (up to 1e-05 allowed)
Greatest relative difference: 0.01153657886452702 at index (3652,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:42:05 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:42:19 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:42:19 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(133.41590642929077, 5.593542368450244)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4066 / 4096 (99.3%)
Greatest absolute difference: 0.005332738161087036 at index (1917,) (up to 1e-05 allowed)
Greatest relative difference: 0.01090978286278136 at index (1917,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:42:21 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:42:24 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:42:24 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(34.452455043792725, 2.6517210577083006)


(<torch.profiler.profiler.profile at 0x7f521803a160>,
 <torch.profiler.profiler.profile at 0x7f520e393bb0>)

In [48]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 34, prefix: 29, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8125 / 8192 (99.2%)
Greatest absolute difference: 0.005303442478179932 at index (2892,) (up to 1e-05 allowed)
Greatest relative difference: 0.010287168273016638 at index (2892,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:42:26 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:42:56 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:42:56 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(291.9512343406677, 13.699558573824788)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8121 / 8192 (99.1%)
Greatest absolute difference: 0.005099058151245117 at index (7969,) (up to 1e-05 allowed)
Greatest relative difference: 0.010028634148983386 at index (7969,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:42:57 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:43:07 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:43:07 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(101.19534492492676, 1.4343835140607553)


(<torch.profiler.profiler.profile at 0x7f520b8e39a0>,
 <torch.profiler.profiler.profile at 0x7f520b21d100>)

In [49]:
gen_and_test(num_field = 34,prefix = 29, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1020 / 1024 (99.6%)
Greatest absolute difference: 0.004173696041107178 at index (455,) (up to 1e-05 allowed)
Greatest relative difference: 0.00809475801906799 at index (455,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:43:08 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:43:15 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:43:15 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(67.44182348251343, 0.8211461653161223)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1019 / 1024 (99.5%)
Greatest absolute difference: 0.004297405481338501 at index (737,) (up to 1e-05 allowed)
Greatest relative difference: 0.008658486688410608 at index (737,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:43:15 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:43:17 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:43:17 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(18.699586391448975, 0.46505127599516527)


(<torch.profiler.profiler.profile at 0x7f520e901760>,
 <torch.profiler.profiler.profile at 0x7f520836e100>)

In [50]:
gen_and_test(num_field = 34,prefix = 29, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2035 / 2048 (99.4%)
Greatest absolute difference: 0.004631459712982178 at index (1554,) (up to 1e-05 allowed)
Greatest relative difference: 0.008890086391828301 at index (1554,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:43:20 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:43:33 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:43:33 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(128.62080097198486, 3.9310997563234196)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2026 / 2048 (98.9%)
Greatest absolute difference: 0.004407107830047607 at index (171,) (up to 1e-05 allowed)
Greatest relative difference: 0.008583714404791342 at index (171,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:43:34 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:43:37 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:43:37 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(33.43014478683472, 1.7043385920430865)


(<torch.profiler.profiler.profile at 0x7f520674d3a0>,
 <torch.profiler.profiler.profile at 0x7f52054f0310>)

In [51]:
gen_and_test(num_field = 34,prefix = 29, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4071 / 4096 (99.4%)
Greatest absolute difference: 0.005630850791931152 at index (2701,) (up to 1e-05 allowed)
Greatest relative difference: 0.01123970615669866 at index (2701,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:43:39 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:44:05 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:44:05 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(260.1369047164917, 8.19910420616452)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4073 / 4096 (99.4%)
Greatest absolute difference: 0.0046433210372924805 at index (210,) (up to 1e-05 allowed)
Greatest relative difference: 0.009140943511240784 at index (210,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:44:06 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:44:13 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:44:13 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(69.98344421386719, 0.6429263359223114)


(<torch.profiler.profiler.profile at 0x7f5203ae44f0>,
 <torch.profiler.profiler.profile at 0x7f520266d8b0>)

In [52]:
gen_and_test(num_field = 34,prefix = 29, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 34, prefix: 29, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 34, prefix: 29


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8120 / 8192 (99.1%)
Greatest absolute difference: 0.006940245628356934 at index (1405,) (up to 1e-05 allowed)
Greatest relative difference: 0.01374078051428504 at index (1405,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:44:16 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:45:09 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:45:09 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(523.1493973731995, 53.50379201205995)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8139 / 8192 (99.4%)
Greatest absolute difference: 0.005733698606491089 at index (1342,) (up to 1e-05 allowed)
Greatest relative difference: 0.01159553182284622 at index (1342,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:45:10 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:45:26 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:45:26 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(158.30207586288452, 12.802591294820331)


(<torch.profiler.profiler.profile at 0x7f52001d2610>,
 <torch.profiler.profiler.profile at 0x7f51ff794cd0>)

In [53]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1015 / 1024 (99.1%)
Greatest absolute difference: 0.004391372203826904 at index (471,) (up to 1e-05 allowed)
Greatest relative difference: 0.008609316408007394 at index (471,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:45:28 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:46:01 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:46:01 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(331.0867238044739, 20.246069455714633)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1012 / 1024 (98.8%)
Greatest absolute difference: 0.004270613193511963 at index (698,) (up to 1e-05 allowed)
Greatest relative difference: 0.008354809516429937 at index (698,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:46:05 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:46:28 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:46:28 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(226.28839492797852, 5.037556480692729)


(<torch.profiler.profiler.profile at 0x7f51fdd979a0>,
 <torch.profiler.profiler.profile at 0x7f51f7d921f0>)

In [54]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2034 / 2048 (99.3%)
Greatest absolute difference: 0.005800366401672363 at index (363,) (up to 1e-05 allowed)
Greatest relative difference: 0.012013538655869781 at index (363,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:46:31 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:47:35 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:47:35 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(637.6942205429077, 95.88031384985217)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2035 / 2048 (99.4%)
Greatest absolute difference: 0.004382610321044922 at index (1387,) (up to 1e-05 allowed)
Greatest relative difference: 0.00870048822508999 at index (1387,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:47:38 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:48:21 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:48:21 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(434.38090085983276, 16.71382417768541)


(<torch.profiler.profiler.profile at 0x7f51f63dae20>,
 <torch.profiler.profiler.profile at 0x7f51f05afca0>)

In [55]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4061 / 4096 (99.1%)
Greatest absolute difference: 0.0042806267738342285 at index (2586,) (up to 1e-05 allowed)
Greatest relative difference: 0.008468877990704836 at index (2586,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:48:28 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:50:35 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:50:35 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1269.2923140525818, 323.11769966333384)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4056 / 4096 (99.0%)
Greatest absolute difference: 0.004278123378753662 at index (3714,) (up to 1e-05 allowed)
Greatest relative difference: 0.008505160686214494 at index (3714,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:50:39 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:52:04 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:52:04 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(852.4554204940796, 132.996860514117)


(<torch.profiler.profiler.profile at 0x7f51f63e4d30>,
 <torch.profiler.profiler.profile at 0x7f51e8d02310>)

In [56]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 110, prefix: 50, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8135 / 8192 (99.3%)
Greatest absolute difference: 0.0065215229988098145 at index (1567,) (up to 1e-05 allowed)
Greatest relative difference: 0.013068608012230955 at index (1567,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:52:15 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:56:26 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:56:26 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2508.982982635498, 1047.0575081689267)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8111 / 8192 (99.0%)
Greatest absolute difference: 0.0049269795417785645 at index (4882,) (up to 1e-05 allowed)
Greatest relative difference: 0.00986396402572547 at index (4882,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:56:34 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 16:59:23 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 16:59:23 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1694.0188145637512, 384.69123365518385)


(<torch.profiler.profiler.profile at 0x7f51f6b8cbe0>,
 <torch.profiler.profiler.profile at 0x7f51e1375700>)

In [57]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1012 / 1024 (98.8%)
Greatest absolute difference: 0.004562497138977051 at index (197,) (up to 1e-05 allowed)
Greatest relative difference: 0.008844477649397447 at index (197,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 16:59:27 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:00:30 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:00:30 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(626.7173099517822, 64.72572884113106)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1012 / 1024 (98.8%)
Greatest absolute difference: 0.004615098237991333 at index (438,) (up to 1e-05 allowed)
Greatest relative difference: 0.009406890902796554 at index (438,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:00:34 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:01:17 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:01:17 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(426.085844039917, 9.548724778278483)


(<torch.profiler.profiler.profile at 0x7f51e1375640>,
 <torch.profiler.profiler.profile at 0x7f51d7ddcd30>)

In [58]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2037 / 2048 (99.5%)
Greatest absolute difference: 0.005117356777191162 at index (117,) (up to 1e-05 allowed)
Greatest relative difference: 0.010618878189822326 at index (117,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:01:23 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:03:30 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:03:30 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1266.2858057022095, 330.1139244442532)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2029 / 2048 (99.1%)
Greatest absolute difference: 0.00493389368057251 at index (88,) (up to 1e-05 allowed)
Greatest relative difference: 0.009997298892328461 at index (88,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:03:34 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:04:59 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:04:59 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(850.3359341621399, 691.6367348834406)


(<torch.profiler.profiler.profile at 0x7f51fb56e520>,
 <torch.profiler.profiler.profile at 0x7f51f9ed1be0>)

In [59]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4070 / 4096 (99.4%)
Greatest absolute difference: 0.00495302677154541 at index (297,) (up to 1e-05 allowed)
Greatest relative difference: 0.010197561963184687 at index (297,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:05:11 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:09:16 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:09:16 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2457.9356694221497, 1901.8331761573052)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4055 / 4096 (99.0%)
Greatest absolute difference: 0.005151152610778809 at index (2446,) (up to 1e-05 allowed)
Greatest relative difference: 0.010599122311923895 at index (2446,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:09:24 3562930:3562930 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:12:05 3562930:3562930 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:12:05 3562930:3562930 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1609.9351024627686, 455.656721815239)


(<torch.profiler.profiler.profile at 0x7f51f863c550>,
 <torch.profiler.profiler.profile at 0x7f51f4195820>)

In [8]:
gen_and_test(num_field = 22 * 5,prefix = 10 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 110, prefix: 50, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 110, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8120 / 8192 (99.1%)
Greatest absolute difference: 0.005399942398071289 at index (2774,) (up to 1e-05 allowed)
Greatest relative difference: 0.010501749615411499 at index (2774,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:36:57 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:45:25 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:45:25 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5076.420528888702, 16091.529130520998)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8139 / 8192 (99.4%)
Greatest absolute difference: 0.00569528341293335 at index (7184,) (up to 1e-05 allowed)
Greatest relative difference: 0.011027116382118878 at index (7184,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:45:39 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:51:08 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:51:08 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(3290.3750133514404, 9771.758506938882)


(<torch.profiler.profiler.profile at 0x7fd57b825b50>,
 <torch.profiler.profiler.profile at 0x7fd4dcf4d8b0>)

In [9]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :1024
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1014 / 1024 (99.0%)
Greatest absolute difference: 0.004258245229721069 at index (393,) (up to 1e-05 allowed)
Greatest relative difference: 0.008456638973254903 at index (393,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:51:14 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:52:37 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:52:37 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(830.8131623268127, 614.1264590047003)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1016 / 1024 (99.2%)
Greatest absolute difference: 0.005850017070770264 at index (4,) (up to 1e-05 allowed)
Greatest relative difference: 0.011828300849102724 at index (4,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:52:39 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:53:01 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:53:01 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(219.8213529586792, 21.95102936659623)


(<torch.profiler.profiler.profile at 0x7fd46858b460>,
 <torch.profiler.profiler.profile at 0x7fd439d7de50>)

In [10]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :2048
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2037 / 2048 (99.5%)
Greatest absolute difference: 0.005620896816253662 at index (1626,) (up to 1e-05 allowed)
Greatest relative difference: 0.011019805571473845 at index (1626,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:53:09 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:55:42 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:55:42 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1527.095651626587, 478.68929813375814)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2030 / 2048 (99.1%)
Greatest absolute difference: 0.004556208848953247 at index (972,) (up to 1e-05 allowed)
Greatest relative difference: 0.009259632198525021 at index (972,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:55:45 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 17:56:25 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 17:56:25 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(403.0729365348816, 19.889734466295295)


(<torch.profiler.profiler.profile at 0x7fd50d82f1c0>,
 <torch.profiler.profiler.profile at 0x7fd408f94310>)

In [11]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4052 / 4096 (98.9%)
Greatest absolute difference: 0.006838709115982056 at index (2319,) (up to 1e-05 allowed)
Greatest relative difference: 0.013781543653726741 at index (2319,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 17:56:39 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:01:36 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:01:36 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2975.36452293396, 1674.0120173368248)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4054 / 4096 (99.0%)
Greatest absolute difference: 0.005009353160858154 at index (633,) (up to 1e-05 allowed)
Greatest relative difference: 0.01032438022607792 at index (633,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:01:40 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:02:56 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:02:56 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(758.4059572219849, 339.9005669212094)


(<torch.profiler.profiler.profile at 0x7fd4006a3130>,
 <torch.profiler.profiler.profile at 0x7fd3f810f940>)

In [12]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 170, prefix: 145, batch :8192
now gen workload of PNN with config: dim: 32, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8130 / 8192 (99.2%)
Greatest absolute difference: 0.005732595920562744 at index (7643,) (up to 1e-05 allowed)
Greatest relative difference: 0.011654435570185114 at index (7643,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:03:22 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:13:10 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:13:10 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5879.927577972412, 51239.153709490325)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8105 / 8192 (98.9%)
Greatest absolute difference: 0.0049295127391815186 at index (462,) (up to 1e-05 allowed)
Greatest relative difference: 0.010010689317545599 at index (462,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:13:17 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:15:42 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:15:42 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1449.394612312317, 155.70080642280558)


(<torch.profiler.profiler.profile at 0x7fd4005f2bb0>,
 <torch.profiler.profiler.profile at 0x7fd35e1a6a30>)

In [13]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 1024, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :1024
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1015 / 1024 (99.1%)
Greatest absolute difference: 0.004903197288513184 at index (607,) (up to 1e-05 allowed)
Greatest relative difference: 0.009717139596912703 at index (607,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:15:50 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:18:23 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:18:23 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1527.7116870880127, 598.5633842023149)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 1015 / 1024 (99.1%)
Greatest absolute difference: 0.004929214715957642 at index (583,) (up to 1e-05 allowed)
Greatest relative difference: 0.009901451828203942 at index (583,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:18:26 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:19:05 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:19:05 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(388.61753940582275, 13.053300227170439)


(<torch.profiler.profiler.profile at 0x7fd35c9a4070>,
 <torch.profiler.profiler.profile at 0x7fd35aaeec10>)

In [14]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 2048, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :2048
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2038 / 2048 (99.5%)
Greatest absolute difference: 0.004391968250274658 at index (280,) (up to 1e-05 allowed)
Greatest relative difference: 0.008756845277709114 at index (280,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:19:19 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:24:19 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:24:19 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2998.169140815735, 2206.580325775328)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 2026 / 2048 (98.9%)
Greatest absolute difference: 0.004844099283218384 at index (627,) (up to 1e-05 allowed)
Greatest relative difference: 0.00989886219204795 at index (627,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:24:23 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:25:32 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:25:32 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(689.4889950752258, 176.28809371714723)


(<torch.profiler.profiler.profile at 0x7fd359215250>,
 <torch.profiler.profiler.profile at 0x7fd357a8a0a0>)

In [15]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 4096, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :4096
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4051 / 4096 (98.9%)
Greatest absolute difference: 0.005629181861877441 at index (780,) (up to 1e-05 allowed)
Greatest relative difference: 0.011143537021092626 at index (780,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:25:57 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:35:20 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:35:20 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(5629.062366485596, 6126.964321565539)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4053 / 4096 (99.0%)
Greatest absolute difference: 0.004902571439743042 at index (1178,) (up to 1e-05 allowed)
Greatest relative difference: 0.010037737543760993 at index (1178,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:35:26 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:37:38 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:37:38 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(1323.8035488128662, 286.23828373665674)


(<torch.profiler.profiler.profile at 0x7fd356915b50>,
 <torch.profiler.profiler.profile at 0x7fd23d394be0>)

In [16]:
gen_and_test(num_field = 34 * 5,prefix = 29 * 5, batch = 8192, dim = 64)

now gen workload of DFM with config: dim: 64, num_field: 170, prefix: 145, batch :8192
now gen workload of PNN with config: dim: 64, num_field: 170, prefix: 145


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8086 / 8192 (98.7%)
Greatest absolute difference: 0.004294335842132568 at index (1515,) (up to 1e-05 allowed)
Greatest relative difference: 0.008662524993717755 at index (1515,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:38:24 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 18:56:35 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 18:56:35 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(10902.739036083221, 17874.122363808026)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 8112 / 8192 (99.0%)
Greatest absolute difference: 0.0057055652141571045 at index (7225,) (up to 1e-05 allowed)
Greatest relative difference: 0.01160141342516359 at index (7225,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-04 18:56:46 3696529:3696529 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-04 19:01:01 3696529:3696529 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-04 19:01:01 3696529:3696529 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(2549.773724079132, 1287.721616166272)


(<torch.profiler.profiler.profile at 0x7fd23b4901f0>,
 <torch.profiler.profiler.profile at 0x7fd23b445040>)

In [17]:
gen_and_test(num_field = 100,prefix = 10, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 10, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 10


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4054 / 4096 (99.0%)
Greatest absolute difference: 0.0057392418384552 at index (2160,) (up to 1e-05 allowed)
Greatest relative difference: 0.011524073828946043 at index (2160,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:10:43 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:12:16 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:12:16 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(928.6528325080872, 98.32132663893255)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4053 / 4096 (99.0%)
Greatest absolute difference: 0.004624009132385254 at index (3579,) (up to 1e-05 allowed)
Greatest relative difference: 0.009068976264107367 at index (3579,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:12:20 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:13:50 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:13:50 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(892.1103930473328, 274.9146124320934)


(<torch.profiler.profiler.profile at 0x7f2826562ca0>,
 <torch.profiler.profiler.profile at 0x7f2814e662b0>)

In [16]:
gen_and_test(num_field = 100,prefix = 20, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 20, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 20


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4064 / 4096 (99.2%)
Greatest absolute difference: 0.005042910575866699 at index (4069,) (up to 1e-05 allowed)
Greatest relative difference: 0.01002970288578899 at index (4069,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:07:36 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:09:10 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:09:10 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(934.1953110694885, 203.33752636662996)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4040 / 4096 (98.6%)
Greatest absolute difference: 0.0040847063064575195 at index (2147,) (up to 1e-05 allowed)
Greatest relative difference: 0.008066335813106777 at index (2147,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:09:14 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:10:38 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:10:38 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(844.2304253578186, 131.24333167340276)


(<torch.profiler.profiler.profile at 0x7f2820fe2130>,
 <torch.profiler.profiler.profile at 0x7f281bcd2c10>)

In [15]:
gen_and_test(num_field = 100,prefix = 30, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 30, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 30


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4049 / 4096 (98.9%)
Greatest absolute difference: 0.005106329917907715 at index (2207,) (up to 1e-05 allowed)
Greatest relative difference: 0.010001154563074925 at index (2207,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:04:38 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:06:10 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:06:10 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(916.4010667800903, 208.3409441561564)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4051 / 4096 (98.9%)
Greatest absolute difference: 0.004858136177062988 at index (1175,) (up to 1e-05 allowed)
Greatest relative difference: 0.009391115667675302 at index (1175,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:06:13 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:07:31 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:07:31 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(772.8156089782715, 134.52108349156333)


(<torch.profiler.profiler.profile at 0x7f2825f877f0>,
 <torch.profiler.profiler.profile at 0x7f2822b709d0>)

In [14]:
gen_and_test(num_field = 100,prefix = 40, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 40, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 40


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4056 / 4096 (99.0%)
Greatest absolute difference: 0.004721164703369141 at index (4057,) (up to 1e-05 allowed)
Greatest relative difference: 0.009484002062562108 at index (4057,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:01:54 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:03:23 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:03:23 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(883.2186031341553, 210.81324130464054)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4052 / 4096 (98.9%)
Greatest absolute difference: 0.004730194807052612 at index (2439,) (up to 1e-05 allowed)
Greatest relative difference: 0.009719832275496762 at index (2439,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:03:27 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:04:33 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:04:33 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(662.7489447593689, 79.72814167302988)


(<torch.profiler.profiler.profile at 0x7f282cee8a90>,
 <torch.profiler.profiler.profile at 0x7f282798c8e0>)

In [13]:
gen_and_test(num_field = 100,prefix = 50, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 50, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 50


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4051 / 4096 (98.9%)
Greatest absolute difference: 0.005218058824539185 at index (3222,) (up to 1e-05 allowed)
Greatest relative difference: 0.010724568191100658 at index (3222,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:59:20 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:00:48 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:00:48 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(887.5364279747009, 88.01392372003517)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4062 / 4096 (99.2%)
Greatest absolute difference: 0.005057841539382935 at index (647,) (up to 1e-05 allowed)
Greatest relative difference: 0.010466054653904773 at index (647,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 22:00:51 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 22:01:50 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 22:01:50 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(582.1899056434631, 55.47309221398659)


(<torch.profiler.profiler.profile at 0x7f28e31755b0>,
 <torch.profiler.profiler.profile at 0x7f28dff47b80>)

In [12]:
gen_and_test(num_field = 100,prefix = 60, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 60, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 60


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4063 / 4096 (99.2%)
Greatest absolute difference: 0.005956172943115234 at index (610,) (up to 1e-05 allowed)
Greatest relative difference: 0.012105391822536952 at index (610,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:56:50 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:58:22 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:58:22 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(921.0778284072876, 129.47397966079262)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4058 / 4096 (99.1%)
Greatest absolute difference: 0.004705488681793213 at index (1678,) (up to 1e-05 allowed)
Greatest relative difference: 0.009108989763477803 at index (1678,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:58:25 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:59:14 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:59:14 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(495.84446907043457, 408.7032263445502)


(<torch.profiler.profiler.profile at 0x7f3064568be0>,
 <torch.profiler.profiler.profile at 0x7f2b1dc86760>)

In [11]:
gen_and_test(num_field = 100,prefix = 70, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 70, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 70


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4054 / 4096 (99.0%)
Greatest absolute difference: 0.005917608737945557 at index (1976,) (up to 1e-05 allowed)
Greatest relative difference: 0.012125246871716596 at index (1976,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:54:30 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:56:01 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:56:01 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(908.3994269371033, 875.8992319614836)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4053 / 4096 (99.0%)
Greatest absolute difference: 0.004890739917755127 at index (798,) (up to 1e-05 allowed)
Greatest relative difference: 0.010185028704159814 at index (798,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:56:03 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:56:44 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:56:44 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(410.7962226867676, 36.36110064703644)


(<torch.profiler.profiler.profile at 0x7f2d3ed4c730>,
 <torch.profiler.profiler.profile at 0x7f2d3e791190>)

In [10]:
gen_and_test(num_field = 100,prefix = 80, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 80, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 80


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4064 / 4096 (99.2%)
Greatest absolute difference: 0.004907697439193726 at index (2246,) (up to 1e-05 allowed)
Greatest relative difference: 0.009944296240328353 at index (2246,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:52:19 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:53:53 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:53:53 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(938.5746169090271, 1109.2481682247299)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4062 / 4096 (99.2%)
Greatest absolute difference: 0.004916787147521973 at index (357,) (up to 1e-05 allowed)
Greatest relative difference: 0.009672804971556747 at index (357,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:53:55 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:54:25 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:54:25 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(297.9980492591858, 56.432528015301294)


(<torch.profiler.profiler.profile at 0x7f2f38d26220>,
 <torch.profiler.profiler.profile at 0x7f2b3b131e20>)

In [9]:
gen_and_test(num_field = 100,prefix = 90, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 90, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 90


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4057 / 4096 (99.0%)
Greatest absolute difference: 0.005376279354095459 at index (3940,) (up to 1e-05 allowed)
Greatest relative difference: 0.010847202352834782 at index (3940,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:50:15 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:51:53 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:51:53 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(984.8476672172546, 1498.3702455634734)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4046 / 4096 (98.8%)
Greatest absolute difference: 0.004737287759780884 at index (2114,) (up to 1e-05 allowed)
Greatest relative difference: 0.009603516393649621 at index (2114,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:51:55 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:52:13 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:52:13 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(183.63572359085083, 13.41921773629906)


(<torch.profiler.profiler.profile at 0x7f30455109a0>,
 <torch.profiler.profiler.profile at 0x7f2f54793ca0>)

In [8]:
gen_and_test(num_field = 100,prefix = 99, batch = 4096, dim = 32)

now gen workload of DFM with config: dim: 32, num_field: 100, prefix: 99, batch :4096
now gen workload of PNN with config: dim: 32, num_field: 100, prefix: 99


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4063 / 4096 (99.2%)
Greatest absolute difference: 0.005261123180389404 at index (637,) (up to 1e-05 allowed)
Greatest relative difference: 0.010339329418927977 at index (637,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:48:24 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:50:00 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:50:00 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(955.861554145813, 1062.348536101831)


/home/yssun/miniconda3/envs/deepctr-torch/lib/python3.9/site-packages/torch/jit/_trace.py:1116: TracerWarning: Output nr 1. of the traced function does not match the corresponding output of the Python function. Detailed error:
Tensor-likes are not close!

Mismatched elements: 4061 / 4096 (99.1%)
Greatest absolute difference: 0.00474393367767334 at index (2629,) (up to 1e-05 allowed)
Greatest relative difference: 0.009823782407966485 at index (508,) (up to 1e-05 allowed)
  _check_trace(
STAGE:2025-02-06 21:50:01 1073686:1073686 ActivityProfilerController.cpp:314] Completed Stage: Warm Up
STAGE:2025-02-06 21:50:09 1073686:1073686 ActivityProfilerController.cpp:320] Completed Stage: Collection
STAGE:2025-02-06 21:50:09 1073686:1073686 ActivityProfilerController.cpp:324] Completed Stage: Post Processing


(76.4285135269165, 8.290420672869914)


(<torch.profiler.profiler.profile at 0x7f30de881400>,
 <torch.profiler.profiler.profile at 0x7f303c10ea90>)